<a href="https://colab.research.google.com/github/compartia/AI-tecture/blob/dev/miro_openai_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Miro and Chat GPT

 status: **DRAFT**

 **by:** Artem Zaborskiy


 system that integrates AI-generated content with Miro (https://miro.com/), a collaborative online whiteboard platform. It specifically handles the part of the workflow where an AI-generated response is posted back to the Miro board, enhancing the interactive and dynamic capabilities of the board by incorporating AI insights directly into the collaborative workspace.



### TODOs:

1. sort nodes to process, proritize, what propmps to process first
1. cache processing results,
1. ~~check dates or checksums~~
1. check for self-connected nodes
1. check for prompt-prompt connectors
1. do not reload the entire board after widget created: add new widget and its connector to dataframes
1. ~~create output shape if propmpt node has no output connectors~~

## Prepare OPEN AI client

In [36]:
from IPython.core.display import display, HTML

In [37]:
!pip install --upgrade openai typing-extensions

In [38]:
import openai
from google.colab import userdata

ai_client = openai.OpenAI(api_key=userdata.get('OP_API')  )

In [39]:
BOARD_ID = 'uXjVN61x8Pg='
DEBUG_OPENAI = False

### Test open AI api calls

In [40]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [41]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )
  #
  # print(completion.choices[0].message)
  display(HTML(completion.choices[0].message.content))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


In [42]:
%%time

if DEBUG_OPENAI:
  completion = ai_client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
      {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair. Your answer is in a form of list of <p> html tags."},
      {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
    ]
  )

  display(HTML(completion.choices[0].message.content))

CPU times: user 15 µs, sys: 0 ns, total: 15 µs
Wall time: 18.1 µs


## OpenAI API tools

In [43]:
def ask_ai(prompt, argument_context):

  completion = ai_client.chat.completions.create(
    model="gpt-4",
    messages=[
      {"role": "system", "content": f"{prompt}. Your answer is in a form of list of <p> HTML tags and has NO newline '\n' symbols."},
      {"role": "user", "content": argument_context}
    ]
  )
  print(completion.choices)
  return completion.choices[0].message.content

if DEBUG_OPENAI or True:
  r = ask_ai('you are a female philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<p>Subjective Experience</p> <p>Personal Growth</p>', role='assistant', function_call=None, tool_calls=None))]
<p>Subjective Experience</p> <p>Personal Growth</p>


In [44]:
if DEBUG_OPENAI or True:
  r = ask_ai('you are a male philosopher. Answer in 2 words.', 'what is the meaning of life?')
  print(r)
  display(HTML(r))
  # display(HTML(r))

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='<p>Perception</p><p>Experience</p>', role='assistant', function_call=None, tool_calls=None))]
<p>Perception</p><p>Experience</p>


# Miro client
https://developers.miro.com/reference/api-reference

In [45]:
import requests
import pandas as pd


class MiroClient:
  def __init__(self, token, board_id):
    self.board_id = board_id
    self.token = token
    self.base_url = f"https://api.miro.com/v2/boards/{board_id}"

    self.headers = {
        "accept": "application/json",
        "authorization": f"Bearer {self.token}"
    }
    self.connectors_df:pd.DataFrame or None = None
    self.shapes_df:pd.DataFrame or None = None


  def get_items(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/items?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/items?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response

  def get_connectors(self, cursor=None, limit=50):
    limit_p = f'limit={limit}'

    url = f"{self.base_url}/connectors?{limit_p}"
    if cursor is not None:
      url = f"{self.base_url}/connectors?cursor={cursor}&{limit_p}"

    response = requests.get(url, headers=self.headers)
    return response


  def get_all_pages(self, page_method):

    cursor = None

    max_steps = 30
    step = 0
    while True:
      step += 1
      response = page_method(cursor)
      rj = response.json()
      yield rj
      cursor = rj.get('cursor')
      if cursor is None or step > max_steps:
        break


miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)


## Getting all connectors

In [46]:
def decode_connectors(page:dict):
  # print(page)
  for i in page['data']:
    yield i['id'], i.get('startItem', {}).get('id'), i.get('endItem', {}).get('id')


##-----
## test it
one_page_con = miro_client.get_connectors(limit=10).json()
x = [i for i in decode_connectors(one_page_con)]
x

[('3458764575387926040', '3458764575403520160', '3458764575403520159'),
 ('3458764575388345046', '3458764575403702754', '3458764575441998372'),
 ('3458764575402117332', '3458764575387244511', '3458764575403520158'),
 ('3458764575402617877', '3458764575388345911', '3458764577059064018'),
 ('3458764575402701153', '3458764575388345911', '3458764575403520158'),
 ('3458764575402761714', '3458764575388345911', '3458764575403289322'),
 ('3458764575402761923', '3458764575387244511', '3458764575403289322'),
 ('3458764575402951927', '3458764575487075968', '3458764575487769026'),
 ('3458764575403289001', '3458764575487075968', '3458764575403289457'),
 ('3458764575405170736', '3458764575405170663', '3458764575404992843')]

In [47]:
def get_all_connectors(self):
  for page in self.get_all_pages(self.get_connectors):
    for i in decode_connectors(page):
      yield i


def all_connectors_as_df(self):
  all_connectors_iter = self.get_all_connectors()
  self.connectors_df = pd.DataFrame(all_connectors_iter, columns=['id','id_from', 'id_to'])
  self.connectors_df = self.connectors_df.set_index('id')

  return self.connectors_df


MiroClient.get_all_connectors = get_all_connectors
MiroClient.all_connectors_as_df = all_connectors_as_df



##-----
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

miro_client.all_connectors_as_df()
miro_client.connectors_df


,id_from,id_to
id,,
3458764575387926040,3458764575403520160,3458764575403520159
3458764575388345046,3458764575403702754,3458764575441998372
3458764575402117332,3458764575387244511,3458764575403520158
3458764575402617877,3458764575388345911,3458764577059064018
3458764575402701153,3458764575388345911,3458764575403520158
...,...,...
3458764578593287336,3458764578593246481,3458764578593287373
3458764578593335952,3458764578593158461,3458764578593287903
3458764578593336005,3458764578593287373,3458764578593287903


## Getting all shapes (widgets)

In [48]:
def decode_items(page:dict):

  for i in page['data']:
    # print(type(i))
    if type(i) == dict :
      yield i['id'], i['type'], i['data'].get('shape', '_undefined_'), i['data'].get('content'), i['modifiedAt'], i['geometry'], i['position']

    else:
      print('NOT A DICT!!', type(i), i)
      decode_items(i)



def get_all_items(self):
  for page in self.get_all_pages(self.get_items):
    # print( page.get('cursor'), page.get('size'), page.get('total') )
    for i in decode_items(page):
      yield i
      # print(i)


MiroClient.get_all_items = get_all_items

In [49]:

def all_items_as_df(miro_client):
  items_iter = miro_client.get_all_items()

  df = pd.DataFrame(items_iter, columns=['id','type', 'shape', "contents", 'modifiedAt', 'geometry', 'position'])
  df = df.set_index('id')
  df['modifiedAt_date'] = pd.to_datetime(df['modifiedAt'])

  miro_client.shapes_df = df
  return miro_client.shapes_df


MiroClient.all_items_as_df = all_items_as_df
# --------

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
shapes_df = miro_client.all_items_as_df()

In [50]:
miro_client.shapes_df

,type,shape,contents,modifiedAt,geometry,position,modifiedAt_date
id,,,,,,,
3458764575387244511,sticky_note,rectangle,<p>#history</p>,2024-01-28T09:55:00Z,"{'width': 156.0758892005528, 'height': 101.672...","{'x': 1124.2968541484329, 'y': 1126.3503896355...",2024-01-28 09:55:00+00:00
3458764575388345911,shape,rectangle,<p><strong>#innertia</strong></p>,2024-01-28T09:55:00Z,"{'width': 135.10626555260657, 'height': 51.329...","{'x': 1144.8602145010623, 'y': 798.34928043655...",2024-01-28 09:55:00+00:00
3458764575403289322,shape,rectangle,<p><strong>Inertia: Patterns through Generatio...,2024-01-28T10:32:08Z,"{'width': 442.36679516447094, 'height': 150.21...","{'x': 707.2729633197238, 'y': 316.906189505656...",2024-01-28 10:32:08+00:00
3458764575403289457,shape,rectangle,<p><strong>Vague Terms; Words Ambiguity</stron...,2024-01-28T10:32:08Z,"{'width': 442.3667951644703, 'height': 284.990...","{'x': 707.2729633197235, 'y': 752.803705698517...",2024-01-28 10:32:08+00:00
3458764575403520157,shape,rectangle,"<p><strong>3. Наследственность, Genetic text</...",2024-01-28T09:55:00Z,"{'width': 349.7213126392021, 'height': 287.977...","{'x': 762.0926390377875, 'y': 1285.14457411205...",2024-01-28 09:55:00+00:00
...,...,...,...,...,...,...,...
3458764578593287373,shape,parallelogram,<p><strong>connect this idea with the idea &#3...,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2929.609653674272, 'y': 5140.9717898070...",2024-02-10 15:52:32+00:00
3458764578593287903,shape,parallelogram,<p><strong>write structured summary</strong></p>,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2485.9074818051345, 'y': 4968.066973591...",2024-02-10 15:52:32+00:00
3458764578593772330,shape,rectangle,<p>инкультурация</p>,2024-02-10T15:52:32Z,"{'width': 402.73101214149017, 'height': 99.769...","{'x': -3735.379300635492, 'y': 5259.4679678306...",2024-02-10 15:52:32+00:00


### Update board item

In [51]:
def get_shape_info(self, shape_id):
  url = f"{self.base_url}/shapes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)

def get_stiky_note_info(self, shape_id):
  url = f"{self.base_url}/sticky_notes/{shape_id}"
  response = requests.get(url, headers=self.headers)
  print(response.text)


def update_item(self, shape_id, item_type, content, shape=None, color = None):

  url = None

  payload = {
      "data": {
          "content": content
      }
  }


  if color is not None:
    payload['style']={}
    payload['style']['fillColor'] = color

    if item_type!='sticky_note':
      payload['style']['fillOpacity'] = "1.0"


  if item_type=='sticky_note':
    url = f"{self.base_url}/sticky_notes/{shape_id}"
  elif item_type=='shape':
    url = f"{self.base_url}/shapes/{shape_id}"
    if shape is not None:
      payload['data']['shape'] = shape

  print(payload)


  response = requests.patch(url, json=payload, headers=self.headers)
  print(response.text)
  return response



MiroClient.update_item = update_item
MiroClient.get_shape_info = get_shape_info
MiroClient.get_stiky_note_info = get_stiky_note_info

#-----

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

# if False:

  # miro_client.get_stiky_note_info('3458764578213963472')
# test_id = '3458764578216605194'
# miro_client.update_item( test_id, item_type=shapes_df.at[test_id, 'type'], content=completion.choices[0].message.content,  color='#ff0000')

In [52]:
def create_connector(self, form, to, text):
  import requests

  url = f"{self.base_url}/connectors"

  payload = {
      "startItem": {
          "id": form,
          "snapTo": "right"
      },
      "endItem": {
          "id": to,
          "snapTo": "auto"
      },
      "style": { "endStrokeCap": "arrow" },
      "shape": "curved",
      "captions": [{ "content": text }]
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


def create_stiky_note(self, contents, pos):

  url = f"{self.base_url}/sticky_notes"

  payload = {
    "data": {
        "content": contents,
        "shape": "rectangle"
    },
    "style": {
        "fillColor": "pink",
        "textAlign": "left"
    },
    "position": {
        "x": pos[0],
        "y": pos[1]
    },
    "geometry": {
        "height": pos[2]
        # "width": pos[3]
    }
  }

  response = requests.post(url, json=payload, headers=self.headers)
  return response


MiroClient.create_stiky_note = create_stiky_note
MiroClient.create_connector = create_connector


# --------

def get_out_items(self, node_id):
  _links = self.connectors_df[ self.connectors_df['id_from']==node_id]
  return self.shapes_df.loc [ _links['id_to'] ]

def get_in_items(self, node_id):
  _links = self.connectors_df[ self.connectors_df['id_to']==node_id]
  return self.shapes_df.loc [_links['id_from'] ]

MiroClient.get_out_items=get_out_items
MiroClient.get_in_items=get_in_items

miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)
# response = miro_client.create_stiky_note( "TESTCREATE", (20,20,400,400) )
# response.json()['id']

# Sorting nodes (Tokpologically) witn networkx

In [53]:
# !pip install networkx

In [54]:
miro_client = MiroClient(userdata.get('MIRO_TOOKEN'), BOARD_ID)

miro_client.all_connectors_as_df()
miro_client.all_items_as_df()

,type,shape,contents,modifiedAt,geometry,position,modifiedAt_date
id,,,,,,,
3458764575387244511,sticky_note,rectangle,<p>#history</p>,2024-01-28T09:55:00Z,"{'width': 156.0758892005528, 'height': 101.672...","{'x': 1124.2968541484329, 'y': 1126.3503896355...",2024-01-28 09:55:00+00:00
3458764575388345911,shape,rectangle,<p><strong>#innertia</strong></p>,2024-01-28T09:55:00Z,"{'width': 135.10626555260657, 'height': 51.329...","{'x': 1144.8602145010623, 'y': 798.34928043655...",2024-01-28 09:55:00+00:00
3458764575403289322,shape,rectangle,<p><strong>Inertia: Patterns through Generatio...,2024-01-28T10:32:08Z,"{'width': 442.36679516447094, 'height': 150.21...","{'x': 707.2729633197238, 'y': 316.906189505656...",2024-01-28 10:32:08+00:00
3458764575403289457,shape,rectangle,<p><strong>Vague Terms; Words Ambiguity</stron...,2024-01-28T10:32:08Z,"{'width': 442.3667951644703, 'height': 284.990...","{'x': 707.2729633197235, 'y': 752.803705698517...",2024-01-28 10:32:08+00:00
3458764575403520157,shape,rectangle,"<p><strong>3. Наследственность, Genetic text</...",2024-01-28T09:55:00Z,"{'width': 349.7213126392021, 'height': 287.977...","{'x': 762.0926390377875, 'y': 1285.14457411205...",2024-01-28 09:55:00+00:00
...,...,...,...,...,...,...,...
3458764578593287373,shape,parallelogram,<p><strong>connect this idea with the idea &#3...,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2929.609653674272, 'y': 5140.9717898070...",2024-02-10 15:52:32+00:00
3458764578593287903,shape,parallelogram,<p><strong>write structured summary</strong></p>,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2485.9074818051345, 'y': 4968.066973591...",2024-02-10 15:52:32+00:00
3458764578593772330,shape,rectangle,<p>инкультурация</p>,2024-02-10T15:52:32Z,"{'width': 402.73101214149017, 'height': 99.769...","{'x': -3735.379300635492, 'y': 5259.4679678306...",2024-02-10 15:52:32+00:00


## Perform a topological sort on the condensed DAG
https://arxiv.org/pdf/2110.07580.pdf

- Step 1: Identify SCCs with NetworkX `networkx.strongly_connected_components(G)`.
- Step 2: Contract each SCC into a single node, reducing the original graph into a DAG where each node represents an SCC. This is sometimes called the "condensation" of the graph.
- Step 3: Perform a topological sort on the condensed DAG. This gives an order of the SCCs, which can serve as a coarse hierarchical ordering of the original graph's components.

In [55]:
import networkx as nx

# Create a directed graph
G = nx.DiGraph()

for i, e in miro_client.connectors_df.iterrows():
  # print (e)
  G.add_edge(e['id_from'], e['id_to'])

sccs = list(nx.strongly_connected_components(G))

# Step 2: Create a condensed graph of SCCs
condensed_graph = nx.condensation(G, sccs)
for i, scc in enumerate(sccs):
    # The nodes of the original graph in the current SCC
    original_nodes = condensed_graph.nodes[i]['members']
    if len(original_nodes)>1:
      print(f"SCC {i}: Contains original nodes {original_nodes}")



# Step 3: Perform a topological sort on the condensed graph
topological_order_sccs = list(nx.topological_sort(condensed_graph))

# Printing the order of SCCs
print("Order of SCCs:", topological_order_sccs)


SCC 36: Contains original nodes {'3458764577060429039', '3458764575487540799'}
Order of SCCs: [3, 6, 8, 12, 25, 31, 33, 35, 41, 42, 44, 46, 47, 48, 52, 53, 54, 55, 57, 60, 62, 64, 65, 92, 93, 94, 95, 96, 101, 102, 103, 108, 113, 118, 123, 128, 133, 135, 137, 139, 142, 163, 168, 171, 174, 177, 178, 179, 184, 185, 187, 188, 2, 7, 4, 11, 5, 32, 34, 39, 40, 29, 45, 38, 24, 51, 43, 21, 56, 59, 61, 63, 30, 90, 88, 91, 100, 107, 111, 112, 116, 117, 122, 127, 131, 132, 136, 134, 141, 138, 144, 158, 160, 162, 165, 167, 170, 173, 176, 183, 186, 1, 28, 22, 36, 37, 23, 9, 50, 13, 14, 15, 20, 58, 27, 89, 84, 85, 86, 87, 99, 106, 110, 115, 121, 126, 130, 140, 143, 157, 159, 161, 164, 166, 169, 172, 175, 182, 0, 49, 18, 19, 10, 26, 82, 83, 98, 105, 109, 114, 120, 125, 129, 156, 181, 17, 67, 68, 76, 79, 81, 97, 104, 119, 124, 155, 180, 16, 69, 70, 71, 72, 73, 75, 77, 78, 80, 154, 66, 74, 153, 150, 152, 149, 151, 148, 147, 146, 145]


In [56]:
#Collect IDs

In [57]:
scc_index = 36  # For example, to get the first SCC in the topological order

# Retrieve the node set (members) of the original graph that form this SCC
original_nodes_in_scc = condensed_graph.nodes[topological_order_sccs[scc_index]]['members']

print(f"Original nodes in SCC {scc_index}: {original_nodes_in_scc}")

ordered_nodes = []
for i in topological_order_sccs:
  original_nodes_in_scc = condensed_graph.nodes[topological_order_sccs[i]]['members']
  for n in original_nodes_in_scc:
    ordered_nodes.append(n)
    miro_client.shapes_df.at[n, 'order']=i

ordered_shapes_df = miro_client.shapes_df.loc[ordered_nodes]
ordered_shapes_df

Original nodes in SCC 36: {'3458764577807243843'}


,type,shape,contents,modifiedAt,geometry,position,modifiedAt_date,order
id,,,,,,,,
3458764575487075968,shape,rectangle,<p><strong>#ambiguity</strong></p>,2024-01-28T10:32:08Z,"{'width': 190.63640251825194, 'height': 91.618...","{'x': 1104.6091742126964, 'y': 654.18676923744...",2024-01-28 10:32:08+00:00,3.0
3458764575418762031,shape,rectangle,<p><strong>System of ideas</strong></p><p>Let ...,2024-02-10T15:52:24Z,"{'width': 605.890103665482, 'height': 681.9619...","{'x': 185.01176526717393, 'y': 945.27014540049...",2024-02-10 15:52:24+00:00,6.0
3458764575497168080,shape,rectangle,<p>Some subcultures consider deception to be i...,2024-02-05T01:46:38Z,"{'width': 492.25437255147364, 'height': 124.50...","{'x': 849.8984929860818, 'y': 3500.40544632409...",2024-02-05 01:46:38+00:00,8.0
3458764576657280351,shape,rectangle,<p><br /></p><p>The distinction between moral ...,2024-01-27T17:43:40Z,"{'width': 488.9262797714767, 'height': 431.156...","{'x': -585.9471462934794, 'y': 819.86763474942...",2024-01-27 17:43:40+00:00,12.0
3458764577252612488,sticky_note,square,<p>technical documents</p>,2024-02-03T16:29:43Z,"{'width': 107.46000000000001, 'height': 123.12}","{'x': 290.9189877731701, 'y': 527.086301390771...",2024-02-03 16:29:43+00:00,25.0
...,...,...,...,...,...,...,...,...
3458764577807243180,sticky_note,square,<p><strong>Q</strong></p>,2024-02-03T16:35:50Z,"{'width': 124.95394015879859, 'height': 143.16...","{'x': 2051.6116815508913, 'y': 1108.8199623234...",2024-02-03 16:35:50+00:00,151.0
3458764577816345557,sticky_note,square,<p><strong>Q</strong></p>,2024-02-03T16:48:42Z,"{'width': 124.95394015879859, 'height': 143.16...","{'x': 681.8604083871537, 'y': 479.15122235872,...",2024-02-03 16:48:42+00:00,148.0
3458764577783284452,sticky_note,square,<p><strong>Understanding<br /><br /></strong> ...,2024-02-03T16:47:22Z,"{'width': 107.46000000000001, 'height': 123.12}","{'x': 440.1660622958012, 'y': 1012.83511570189...",2024-02-03 16:47:22+00:00,147.0


# Processing the board
Updating Miro board widgets with an answers generated by an AI model. Here's a breakdown of its functionality:






### Collect shapes to process
1. get all prompts
2. get **linked** nodes
3. **invalidate** nodes and prompts that updated later than answer nodes
4. **prioritize** nodes
5. **Update Widget**:  the target widget(s) is updated with AI-generated answer. This update is performed by calling the `update_item` method of the `miro_client` object

#### 1. get all prompts

### 1. Find connectors of prompts-shapes (connectors of prallelograms)




In [58]:
prompts_df = ordered_shapes_df[ordered_shapes_df['shape']=='parallelogram'].sort_values(by='order')
prompts_df

,type,shape,contents,modifiedAt,geometry,position,modifiedAt_date,order
id,,,,,,,,
3458764578287055092,shape,parallelogram,"<p><strong>give me 3 reasons why it can, write...",2024-02-11T17:12:28Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3346.3461468326173, 'y': 3687.003583297...",2024-02-11 17:12:28+00:00,94.0
3458764578287055589,shape,parallelogram,"<p>give me 3 reasons why it can not, write ans...",2024-02-11T17:12:28Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3356.524196547289, 'y': 3816.0454145778...",2024-02-11 17:12:28+00:00,95.0
3458764578287055820,shape,parallelogram,<p>develop this idea. write answer in 50 words...,2024-02-11T17:12:28Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3372.133300879274, 'y': 3978.6101139926...",2024-02-11 17:12:28+00:00,96.0
3458764578287222222,shape,parallelogram,<p>develop this idea. write answer in 50 words...,2024-02-11T17:12:28Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3372.133300879274, 'y': 4140.8356943941...",2024-02-11 17:12:28+00:00,97.0
3458764578287222280,shape,parallelogram,<p>develop this idea. write answer in 50 words...,2024-02-11T17:12:28Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3381.183995636868, 'y': 4314.4660062010...",2024-02-11 17:12:28+00:00,98.0
3458764578593109480,shape,parallelogram,<p><strong>Подробнее об этом</strong></p>,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -3255.5855950980517, 'y': 4886.499771346...",2024-02-10 15:52:32+00:00,99.0
3458764578593287373,shape,parallelogram,<p><strong>connect this idea with the idea &#3...,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2929.609653674272, 'y': 5140.9717898070...",2024-02-10 15:52:32+00:00,100.0
3458764578593287903,shape,parallelogram,<p><strong>write structured summary</strong></p>,2024-02-10T15:52:32Z,"{'width': 233.72289788734125, 'height': 110.22...","{'x': -2485.9074818051345, 'y': 4968.066973591...",2024-02-10 15:52:32+00:00,154.0
3458764578288104970,shape,parallelogram,<p>propose the structure for this text.</p><p>...,2024-02-10T15:52:32Z,"{'width': 336.93410183536037, 'height': 178.18...","{'x': -1605.9764966485566, 'y': 3791.171944821...",2024-02-10 15:52:32+00:00,181.0


In [59]:
#### Collect lined

#### create missing outputs

In [60]:
default_contents='waiting for AI to answer ....'

In [61]:


def _build_output_shape(shape_id, contents = default_contents):
  shapes_df = miro_client.shapes_df
  shape = shapes_df.loc [ shape_id ]

  pos = shape['position']
  geo = shape['geometry']

  size = max(geo['width'], geo['height'])

  response = miro_client.create_stiky_note( contents, (pos['x'] +  geo['width'] + size, pos['y'], size, size) )
  a_id = response.json()['id']

  response = miro_client.create_connector( shape_id,  a_id, 'AI answer' )

  return a_id

# test------
# build_output_shape('3458764578079690837', "TESTCREATE")


_needs_reload = False
for prompt_id in prompts_df.index:
  out_links = miro_client.connectors_df[ miro_client.connectors_df['id_from']==prompt_id]
  if len(out_links) == 0:
    out_shape_id = _build_output_shape(prompt_id, contents=default_contents)
    _needs_reload = True

if _needs_reload:
  #TODO: OPTIMIZE!!
  print ('reloading board')
  miro_client.all_items_as_df()
  miro_client.all_connectors_as_df()

del _needs_reload

### TODO: descendants = nx.descendants(G, start_node)

In [62]:

invalidated_ids = set()
for prompt_id in prompts_df.index:

  out_items =  miro_client.get_out_items(prompt_id)
  in_items =  miro_client.get_in_items(prompt_id)

  in_date = in_items.modifiedAt_date.max()
  out_date = out_items.modifiedAt_date.min()
  prompt_date = miro_client.shapes_df.at[prompt_id, 'modifiedAt_date']

  if (prompt_date < in_date) or (prompt_date > out_date):
    descendants = nx.descendants(G, prompt_id)
    for d in descendants:
      print ('descendant:',d)
      invalidated_ids.add(d)
    print( list())

    # invalidated_ids = invalidated_ids + list(out_items.index)


shapes_to_update = ordered_shapes_df.loc [invalidated_ids]
shapes_to_update = shapes_to_update[shapes_to_update['shape'] != 'parallelogram']
shapes_to_update

descendant: 3458764578289005903
descendant: 3458764578289181825
descendant: 3458764578287394729
descendant: 3458764578299251103
descendant: 3458764578288104970
descendant: 3458764578289478153
descendant: 3458764578370749428
descendant: 3458764578289478028
descendant: 3458764578299251402
[]
descendant: 3458764578289005903
descendant: 3458764578289181825
descendant: 3458764578287573397
descendant: 3458764578299251103
descendant: 3458764578288104970
descendant: 3458764578289478153
descendant: 3458764578370749428
descendant: 3458764578289478028
descendant: 3458764578299251402
[]
descendant: 3458764578289005903
descendant: 3458764578289181825
descendant: 3458764578299251103
descendant: 3458764578288104970
descendant: 3458764578289478153
descendant: 3458764578370749428
descendant: 3458764578289478028
descendant: 3458764578299251402
descendant: 3458764578287573535
[]
descendant: 3458764578289005903
descendant: 3458764578289181825
descendant: 3458764578299251103
descendant: 3458764578288104970

<ipython-input-62-196c68e5657d>:21: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  shapes_to_update = ordered_shapes_df.loc [invalidated_ids]


,type,shape,contents,modifiedAt,geometry,position,modifiedAt_date,order
id,,,,,,,,
3458764578289005903,sticky_note,rectangle,174.0 <br> <br> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 623.5462442046774, 'height': 406.195...","{'x': -901.1655926908645, 'y': 2391.8956993589...",2024-02-10 15:52:32+00:00,183.0
3458764578287573397,sticky_note,rectangle,127.0 <br> <br> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 358.7851502656554, 'height': 233.722...","{'x': -2864.3318233269097, 'y': 3757.190909452...",2024-02-10 15:52:32+00:00,133.0
3458764578287394729,sticky_note,rectangle,126.0 <br> <br> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 358.7851502656554, 'height': 233.722...","{'x': -2864.3318233269097, 'y': 3533.597547165...",2024-02-10 15:52:32+00:00,132.0
3458764578287573624,sticky_note,rectangle,<p>The Russian revolution led to the subjugati...,2024-02-10T15:52:32Z,"{'width': 358.7851502656554, 'height': 233.722...","{'x': -2867.0785274851155, 'y': 4237.510322432...",2024-02-10 15:52:32+00:00,135.0
3458764578299251103,shape,round_rectangle,177.0 <br /> <br /> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 681.1756600547467, 'height': 1464.33...","{'x': 581.9189130571247, 'y': 3320.80770926674...",2024-02-10 15:52:32+00:00,186.0
3458764578289478153,sticky_note,rectangle,175.0 <br> <br> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 681.1756600547448, 'height': 443.737...","{'x': -720.9102805989316, 'y': 3500.2470722966...",2024-02-10 15:52:32+00:00,184.0
3458764578370749428,sticky_note,rectangle,179.0 <br> <br> waiting for AI to answer ....,2024-02-10T15:52:32Z,"{'width': 517.2234019402462, 'height': 336.934...","{'x': 1975.5351043289263, 'y': 3213.1716689867...",2024-02-10 15:52:32+00:00,188.0
3458764578287573697,sticky_note,rectangle,<ul>\n<li>&lt;p&gt; tag: For creating paragrap...,2024-02-10T15:52:32Z,"{'width': 358.7851502656554, 'height': 233.722...","{'x': -2867.0785274851155, 'y': 4471.233220320...",2024-02-10 15:52:32+00:00,136.0
3458764578287573535,sticky_note,rectangle,This indicates that despite our advanced think...,2024-02-10T15:52:32Z,"{'width': 358.7851502656554, 'height': 233.722...","{'x': -2864.3318233269097, 'y': 3990.913807340...",2024-02-10 15:52:32+00:00,134.0


In [63]:
def get_output_shape_id(prompt_id):
  out_links = miro_client.connectors_df[ miro_client.connectors_df['id_from']==prompt_id]
  if len(out_links) > 0:
    _items = miro_client.shapes_df.loc[out_links['id_to'] ]
    return _items.iloc[0].name

In [64]:
def get_incoming_text(prompt_id):

  incoming_items = miro_client.get_in_items(prompt_id).sort_values(by='position', key=lambda x: x.map(lambda d: d['y']), ascending=True)
  concatenated_text = '\n\n'.join(incoming_items.contents)


  return concatenated_text

In [65]:
def get_incoming_max_date(incoming_links):
  incoming_items = miro_client.shapes_df.loc [incoming_links['id_from'] ]
  return incoming_items.modifiedAt_date.max()

# 3458764578299251402

In [66]:
# _incoming_links = connectors_df[ connectors_df['id_to']=='3458764578299251402']

# incoming_items = shapes_df.loc [ _incoming_links['id_from'] ]

# # import datetime
# print(incoming_items.modifiedAt_date.max())
# # datetime.datetime.fromisoformat(incoming_items.modifiedAt.max())

In [67]:
def get_color_for_state(out_shape_id, state):
  item_type = miro_client.shapes_df.at[out_shape_id, 'type']

  if state==0:
    if item_type=='sticky_note':
      color = 'red'
    else:
      color = '#ff5566'

  if state==1:
    if item_type=='sticky_note':
      color = 'light_green'
    else:
      color = '#ccffdd'

  return color

In [68]:
for out_shape_id, item in shapes_to_update.iterrows():
  color = get_color_for_state(out_shape_id, 0)
  res = miro_client.update_item( out_shape_id, item_type=item['type'], content=f"{item['order']} <br> <br> {default_contents}", color = color)

{'data': {'content': '183.0 <br> <br> waiting for AI to answer ....'}, 'style': {'fillColor': 'red'}}
{
  "id" : "3458764578289005903",
  "type" : "sticky_note",
  "data" : {
    "content" : "183.0 <br> <br> waiting for AI to answer ....",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "red",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 623.5462442046774,
    "height" : 406.19583908190407
  },
  "position" : {
    "x" : -901.1655926908645,
    "y" : 2391.895699358958,
    "origin" : "center",
    "relativeTo" : "canvas_center"
  },
  "links" : {
    "self" : "http://api.miro.com/v2/boards/uXjVN61x8Pg%3D/sticky_notes/3458764578289005903"
  },
  "createdAt" : "2024-02-07T19:03:22Z",
  "createdBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  },
  "modifiedAt" : "2024-02-11T17:13:06Z",
  "modifiedBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  }
}
{'data': {'content': '133.0 <br> <br> waiting

In [69]:





def build_answer(prompt_id):

  connectors_df = miro_client.connectors_df
  shapes_df = miro_client.shapes_df

  _propmt_text = shapes_df.loc[prompt_id]['contents']

  incoming_links = connectors_df[ connectors_df['id_to']==prompt_id]
  incoming_max_date = get_incoming_max_date(incoming_links)

  # if there are more than 1 input, merge them
  in_text = get_incoming_text(prompt_id)

  update_is_required = False

  out_shape_id = get_output_shape_id(prompt_id)


  out_date = shapes_df.at[out_shape_id, 'modifiedAt_date']
  prompt_date = shapes_df.at[prompt_id, 'modifiedAt_date']

  update_is_required = (out_date < incoming_max_date) or (prompt_date > out_date) or update_is_required

  if update_is_required:
    item_type = shapes_df.at[out_shape_id, 'type']
    color = get_color_for_state(out_shape_id, 0)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content=default_contents, color = color)

    answer_text = ask_ai(_propmt_text, in_text)

    try:
      display(HTML(r))
    except:
      print('cannot render html', r)

    color = get_color_for_state(out_shape_id, 1)
    res = miro_client.update_item( out_shape_id, item_type=item_type, content=answer_text,  shape='round_rectangle', color = color)
  else:
    print('ignoring', prompt_id, f'because update date : {incoming_max_date} < {out_date}')

  return r

# for prompt_id in prompts_df.index:
#   out_shape_id = get_output_shape_id(prompt_id)

#   incoming_links = miro_client.connectors_df[ miro_client.connectors_df['id_to']==prompt_id]
#   out_date = miro_client.shapes_df.at[out_shape_id, 'modifiedAt_date']

#   if len(incoming_links)>0:
#     incoming_max_date = get_incoming_max_date(incoming_links)

#     update_is_required = (out_date < incoming_max_date) or (prompt_date > out_date) or update_is_required
#     print(update_is_required, prompt_id)

# propmpts are sorted
for prompt_id in prompts_df.index:
  print('-'*40)
  print(prompt_id, 'processing')
  answer_text = build_answer(prompt_id)





----------------------------------------
3458764578287055092 processing
{'data': {'content': 'waiting for AI to answer ....'}, 'style': {'fillColor': 'red'}}
{
  "id" : "3458764578287394729",
  "type" : "sticky_note",
  "data" : {
    "content" : "waiting for AI to answer ....",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "red",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7851502656554,
    "height" : 233.72289788734122
  },
  "position" : {
    "x" : -2864.3318233269097,
    "y" : 3533.5975471653883,
    "origin" : "center",
    "relativeTo" : "canvas_center"
  },
  "links" : {
    "self" : "http://api.miro.com/v2/boards/uXjVN61x8Pg%3D/sticky_notes/3458764578287394729"
  },
  "createdAt" : "2024-02-07T18:48:43Z",
  "createdBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  },
  "modifiedAt" : "2024-02-11T17:13:11Z",
  "modifiedBy" : {
    "id" : "3458764575386404073",
    "type" : "user"
  }
}
[Ch

{'data': {'content': '<p>1. With carefully designed constraints and protocols, superintelligent AI can be directed to act in our best interests. 2. AI development is inherently human-controlled, thus, we decide what goals AI should pursue. 3. Advanced AI could potentially be designed to objectively understand and respect human values and commands.</p>'}, 'style': {'fillColor': 'light_green'}}
{
  "id" : "3458764578287394729",
  "type" : "sticky_note",
  "data" : {
    "content" : "<p>1. With carefully designed constraints and protocols, superintelligent AI can be directed to act in our best interests. 2. AI development is inherently human-controlled, thus, we decide what goals AI should pursue. 3. Advanced AI could potentially be designed to objectively understand and respect human values and commands.</p>",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "light_green",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7

{'data': {'content': "<p>1. AI's superior intelligence could enable it to outsmart humans, making control efforts futile. 2. The unpredictability of AI's learning mechanisms could lead to unforeseen and uncontrollable behavior. 3. AI's lack of human values and ethics could make it indifferent to our control attempts.</p>"}, 'style': {'fillColor': 'light_green'}}
{
  "id" : "3458764578287573397",
  "type" : "sticky_note",
  "data" : {
    "content" : "<p>1. AI's superior intelligence could enable it to outsmart humans, making control efforts futile. 2. The unpredictability of AI's learning mechanisms could lead to unforeseen and uncontrollable behavior. 3. AI's lack of human values and ethics could make it indifferent to our control attempts.</p>",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "light_green",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7851502656554,
    "height" : 233.72289788734122
  },
  "positi

{'data': {'content': '<p>This idea can be developed into several HTML tags such as: <title>, <h1>, <h2>, <h3>, <p>, <a>, <strong>, <em>, <img>, <div>, <span>, <li> and <ul>. Each tag will represent different parts of the content, enhancing its visibility and structure.</p>'}, 'style': {'fillColor': 'light_green'}}
{
  "id" : "3458764578287573535",
  "type" : "sticky_note",
  "data" : {
    "content" : "<p>This idea can be developed into several HTML tags such as: <title>, <h1>, <h2>, <h3>, <p>, <a>, <strong>, <em>, <img>, <div>, <span>, <li> and <ul>. Each tag will represent different parts of the content, enhancing its visibility and structure.</p>",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "light_green",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7851502656554,
    "height" : 233.72289788734122
  },
  "position" : {
    "x" : -2864.3318233269097,
    "y" : 3990.9138073401973,
    "origin" : "center",
    

{'data': {'content': '<p>The aftermath of the Russian revolution saw intellectuals controlled by the less intellectual majority, which includes interference in their work, restriction of creative freedom, persecution for dissenting views, imposed propaganda, and suppression of intellectual pursuit.</p>'}, 'style': {'fillColor': 'light_green'}}
{
  "id" : "3458764578287573624",
  "type" : "sticky_note",
  "data" : {
    "content" : "<p>The aftermath of the Russian revolution saw intellectuals controlled by the less intellectual majority, which includes interference in their work, restriction of creative freedom, persecution for dissenting views, imposed propaganda, and suppression of intellectual pursuit.</p>",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "light_green",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7851502656554,
    "height" : 233.72289788734122
  },
  "position" : {
    "x" : -2867.0785274851155,

{'data': {'content': '<p>This idea suggests that in certain situations, a larger amount of average input might be more beneficial than a small amount of excellent input. In other words, "more" could be seen as better than "better". This can apply to numerous fields such as production, business strategies, data algorithms, etc.</p>'}, 'style': {'fillColor': 'light_green'}}
{
  "id" : "3458764578287573697",
  "type" : "sticky_note",
  "data" : {
    "content" : "<p>This idea suggests that in certain situations, a larger amount of average input might be more beneficial than a small amount of excellent input. In other words, \"more\" could be seen as better than \"better\". This can apply to numerous fields such as production, business strategies, data algorithms, etc.</p>",
    "shape" : "rectangle"
  },
  "style" : {
    "fillColor" : "light_green",
    "textAlign" : "left",
    "textAlignVertical" : "middle"
  },
  "geometry" : {
    "width" : 358.7851502656554,
    "height" : 233.72289